# About Dataset

<center><img src="https://www.freepnglogos.com/uploads/spotify-logo-png/image-gallery-spotify-logo-21.png" width="400"/><center/>

## Audio Features 

- **duration_ms** - The duration of the track in milliseconds.



- **key** - The estimated overall key of the track. Integers map to pitches using standard Pitch Class notation . E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. If no key was detected, the value is -1.



- **mode** - Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0.



- **time_signature** - An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure).



- **acousticness** - A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic.



- **danceability** - Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable.



- **energy** - Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.



- **instrumentalness** - Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0.



- **liveness** - Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. The distribution of values for this feature look like this: Liveness distribution.



- **loudness** - The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db.



- **speechiness** - Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks.



- **valence** - A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).



- **tempo** - The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration.



- **id** - The Spotify ID for the track.



- **popularity** - The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time.



- More information can be found [here](https://www.kaggle.com/yamaerenay/spotify-dataset-19212020-160k-tracks)

# Importing Dependencies

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import seaborn as sns
import warnings

from matplotlib import pyplot as plt

In [ ]:
warnings.filterwarnings('ignore')
plt.rcParams['figure.dpi'] = 200
%matplotlib inline

# Importing Data

In [ ]:
tracks_data = pd.read_csv('/kaggle/input/spotify-dataset-19212020-160k-tracks/tracks.csv')

In [ ]:
tracks_data.head()

## Data  Cleaning

## Checking for Null Values

In [ ]:
#checking for null values
tracks_data.isnull().sum()

In [ ]:
# Dropping rows where name is null
tracks_data.dropna(subset=['name'], axis=0, inplace=True) 

**In version 15 of this dataset, 'tracks.csv' contains tracks which have null values in audio features. Also, there are some redundant tracks (their IDs are unique but they have similar audio features and same track name. This issue has been discussed [here](https://www.kaggle.com/yamaerenay/spotify-dataset-19212020-160k-tracks/discussion/239435)**

# Rows where audio features are null

In [ ]:
tracks_data[tracks_data['name'] == 'Pause Track']

# Remoivng Duplicate rows

In [ ]:
cols = tracks_data.columns.tolist()
to_remove = ['id', 'popularity', 'duration_ms', 'explicit',
             'release_date', 'time_signature', 'key', 'mode']
reqd_cols = list(set(cols).difference(set(to_remove)))

#Dropping Duplicates
tracks_data.drop_duplicates(subset=reqd_cols, keep=False, ignore_index=True, inplace=True)
i = tracks_data[tracks_data.name == 'Pause Track'].index
tracks_data.drop(i, axis=0, inplace=True)

## Descriptive Statistics

In [ ]:
tracks_data.info()

In [ ]:
cat_cols = ['explicit', 'key', 'mode', 'time_signature', 'release_date']
tracks_data.drop(cat_cols, axis=1, inplace=False).describe().transpose().sort_index()

In [ ]:
#setting release date as the index
tracks_data.set_index('release_date', inplace=True)
tracks_data.index = pd.to_datetime(tracks_data.index)
tracks_data.sort_index(inplace=True)

#Extracting only year from index
tracks_data['release_year'] = tracks_data.index.year.values

#Converting duration_ms(milliseocnds to seconds)
tracks_data['duration_s'] = tracks_data['duration_ms']/1000

## Data Visualization

In [ ]:
def year_to_decade(year):
    decade = (year//10) * 10
    decade = f'{decade}s'
    return decade

tracks_data['decade'] = tracks_data['release_year'].apply(lambda x: year_to_decade(x))

In [ ]:
fig = plt.figure(figsize = (20,8),facecolor = 'white')
gs = fig.add_gridspec(1,1)
ax = fig.add_subplot(gs[0,0])

ax.text(-0.5, 115000, 
         'Songs Released(By Decade)', 
         fontsize=25, 
         fontweight='bold', 
         fontfamily='monospace'
        )

ax.text(-0.5, 109000, 
         'Most of the songs (19% of total songs in the dataset) are released in the 90s.', 
         fontsize=15, 
         fontweight='light', 
         fontfamily='monospace'
        )

ax.grid(color='black', linestyle=':', axis='y', zorder=0,  dashes=(1,5))


colormap = ["lightgreen" for _ in range(12)]
colormap[8] = "#0ddb22"

sns.countplot(data = tracks_data, x ='decade', ax = ax, palette = colormap, alpha = 1, zorder = 2)


for direction in ['top','right','left']:
    ax.spines[direction].set_visible(False)

ax.set_xlabel('Decade', fontsize = 14, fontweight = 'bold')
ax.tick_params(axis = 'x', labelsize=14)
ax.tick_params(axis = 'y', length=0, labelsize=13)
ax.set_ylabel('',)
plt.show()

In [ ]:
decade_data = pd.DataFrame(tracks_data['decade'].value_counts().sort_index()).rename({'decade': 'count'}, axis=1)

decade_data

**There is only one song released in 1900. That's why, it is not visible in the plot.**

In [ ]:
tracks_data[tracks_data['decade'] == '1900s']

In [ ]:
fig = plt.figure(figsize=(10, 8), facecolor='white')
gs = fig.add_gridspec(1, 1)
ax= fig.add_subplot(gs[0, 0])

ax.text(0.2, 580000, 
         'Non Explicit vs Explicit', 
         fontsize=25, 
         fontweight='bold', 
         fontfamily='monospace'
        )

ax.text(0.2, 550000, 
         '96% of the songs have no explicit content.', 
         fontsize=15, 
         fontweight='light', 
         fontfamily='monospace'
        )

ax.grid(color='black', linestyle=':', axis='y', zorder=0,  dashes=(1,5))

sns.countplot(x = 'explicit', data = tracks_data, zorder=2, ax = ax, alpha=1, palette=['#28b0eb', '#ff6161'])

for direction in ['top','right','left']:
    ax.spines[direction].set_visible(False)


ax.set_xlabel('',)
ax.set_xticklabels(['Non Explicit', 'Explicit'], fontweight='bold')
ax.tick_params(axis='x', labelsize=14)
ax.tick_params(axis = 'y', length=0, labelsize=12)
ax.set_ylabel('',)

plt.show()

In [ ]:
explicit_tracks = tracks_data[tracks_data['explicit']==1]

fig = plt.figure(figsize=(20, 8), facecolor='white')
gs = fig.add_gridspec(1, 1)
ax = fig.add_subplot(gs[0, 0])

ax.text(-0.5, 14000, 
         'Explicit songs released (By Decade)', 
         fontsize=25, 
         fontweight='bold', 
         fontfamily='monospace'
        )

ax.text(-0.5, 13000, 
         'Most of the explicit songs are released between 2010 and 2020', 
         fontsize=15, 
         fontweight='light', 
         fontfamily='monospace'
        )

ax.grid(color='black', linestyle=':', axis='y', zorder=0,  dashes=(1,5))

colormap = ["#ff6161" for _ in range(12)]
colormap[9] = "#c94040"

sns.countplot(x = 'decade', data = explicit_tracks, zorder=2, ax = ax, alpha=1, palette=colormap)

for direction in ['top','right','left']:
    ax.spines[direction].set_visible(False)


ax.set_xlabel('Decade', fontsize = 14, fontweight = 'bold')
ax.tick_params(axis='x', labelsize=14)
ax.tick_params(axis = 'y', length=0, labelsize=13)
ax.set_ylabel('',)
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 8), facecolor='white')
gs = fig.add_gridspec(1, 1)

ax = fig.add_subplot(gs[0, 0])

ax.text(0, 400000, 
         'Modality Distribution', 
         fontsize=25, 
         fontweight='bold', 
         fontfamily='monospace'
        )

ax.grid(color='black', linestyle=':', axis='y', zorder=0,  dashes=(1,5))

sns.countplot(x = 'mode', data = tracks_data, zorder=2, ax = ax, alpha=1, palette=['#03fcfc', '#fcda00'])

for direction in ['top','right','left']:
    ax.spines[direction].set_visible(False)


ax.set_xticklabels(['Minor', 'Major'], fontweight='bold', fontsize=13)
ax.set_xlabel('',)
ax.tick_params(axis = 'y', length=0, labelsize=12)
ax.set_ylabel('',)

plt.show()

# Encoding keys according to Pitch Class Notation

In [ ]:
#Encoding keys according to Pitch Class Notation
key_map = {}

key_list = ['C(B♯, etc.)', 'C♯, D♭', 'D(Cx, etc.)',
            'D♯, E♭', 'E(F♭, etc.)', 'F(E♯, etc.)',
            'F♯, G♭', 'G(Fx, etc.)', 'G♯, A♭', 'A(Gx, etc.)',
            'A♯, B♭', 'B(C♭, etc.)']

for i, key in enumerate(key_list):
    key_map[i] = key

tracks_data['key_map'] = tracks_data['key'].replace(key_map)
print(key_map)

In [ ]:
fig = plt.figure(figsize=(20, 8), facecolor='white')
gs = fig.add_gridspec(1, 1)

ax = fig.add_subplot(gs[0, 0])

ax.text(3.5, 80000, 
         'Distribution of Keys', 
         fontsize=25, 
         fontweight='bold', 
         fontfamily='monospace'
        )

ax.grid(color='black', linestyle=':', axis='y', zorder=0,  dashes=(1,5))

sns.countplot(x = 'key_map', data = tracks_data, zorder=2, ax = ax, alpha=1, palette='Set3')

for direction in ['top','right','left']:
    ax.spines[direction].set_visible(False)


ax.tick_params(axis='y', labelsize=14)
ax.tick_params(axis='x', labelsize=15, rotation=-40)
ax.set_xlabel('',)
ax.tick_params(axis = 'y', length=0)
ax.set_ylabel('',)

plt.show()

In [ ]:
fig = plt.figure(figsize=(20, 8), facecolor='white')
gs = fig.add_gridspec(1, 1)
ax = fig.add_subplot(gs[0, 0])

ax.text(0.5, 550000, 
         'Distribution of Time Signature', 
         fontsize=25, 
         fontweight='bold', 
         fontfamily='monospace'
        )

ax.text(-0.5, 520000, 
         '85% of the songs have a time signature of 4(All the notes in each bar must add up to 4 quarter notes)', 
         fontsize=15, 
         fontweight='light', 
         fontfamily='monospace'
        )


ax.grid(color='black', linestyle=':', axis='y', zorder=0,  dashes=(1,5))

colormap = ['#e292e8'] * 5
colormap[3] = '#d712e6'

sns.countplot(x = 'time_signature', data = tracks_data, zorder=2, ax = ax, alpha=1, palette=colormap)

for direction in ['top','right','left']:
    ax.spines[direction].set_visible(False)


ax.tick_params(axis='y', labelsize=14)
ax.tick_params(axis='x', labelsize=14)
ax.set_xlabel('Time Signature',fontsize=15, fontweight='bold')
ax.set_ylabel('')
plt.show()

In [ ]:
time_data = pd.DataFrame(tracks_data['time_signature'].value_counts().sort_index()).reset_index().rename({'index':'time_signature', 'time_signature': 'count'}, axis=1)

time_data

In [ ]:
audio_features = ['danceability', 'energy', 'speechiness', 'acousticness',
                  'instrumentalness', 'liveness', 'valence']

colormap = ['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02','#a6761d']

audio_data = pd.DataFrame(tracks_data.groupby('release_year')[audio_features].mean().sort_index()).reset_index()

fig = plt.figure(figsize=(20, 8), facecolor='white')
gs = fig.add_gridspec(1, 1)
ax = fig.add_subplot(gs[0, 0])

ax.text(1922, 1.1, 
         'Year Wise Distribution of Audio Features', 
         fontsize=25, 
         fontweight='bold', 
         fontfamily='monospace'
        )

for feature, color in zip(audio_features, colormap):
    sns.lineplot(data=audio_data.iloc[1:, ], x='release_year', y=feature, color=color, ax=ax)
    
for direction in ['top','right','left']:
    ax.spines[direction].set_visible(False)
    
ax.tick_params(axis='y', labelsize=14)
ax.tick_params(axis='x', labelsize=14)
ax.tick_params(axis = 'y', length=0)  
ax.set_xlabel('Year', fontsize=15, fontweight='bold')
ax.set_ylabel('')
ax.legend(audio_features, loc='upper center', fontsize=11)

plt.show()

In [ ]:
loudness_data = pd.DataFrame(tracks_data.groupby('release_year')['loudness'].mean().sort_index()).reset_index()

fig = plt.figure(figsize=(20, 8), facecolor='white')
gs = fig.add_gridspec(1, 1)
ax = fig.add_subplot(gs[0, 0])

ax.text(1920, -6, 
         'Loudness Distribution (Year Wise)', 
         fontsize=25, 
         fontweight='bold', 
         fontfamily='monospace'
        )

sns.lineplot(x='release_year', y='loudness', data=loudness_data.iloc[1:,], ax=ax, color='#ccba1d')

for direction in ['top','right','left']:
    ax.spines[direction].set_visible(False)

ax.tick_params(axis='y', labelsize=14)
ax.tick_params(axis='x', labelsize=14)
ax.tick_params(axis = 'y', length=0)  
ax.set_xlabel('Year', fontsize=15, fontweight='bold')
ax.set_ylabel('')
    
plt.show()

In [ ]:
tempo_data = pd.DataFrame(tracks_data.groupby('release_year')['tempo'].mean().sort_index()).reset_index()

fig = plt.figure(figsize=(20, 8), facecolor='white')
gs = fig.add_gridspec(1, 1)
ax = fig.add_subplot(gs[0, 0])

ax.text(1920, 125, 
         'Tempo Distribution (Year Wise)', 
         fontsize=25, 
         fontweight='bold', 
         fontfamily='monospace'
        )

sns.lineplot(x='release_year', y='tempo', data=tempo_data.iloc[1:, ], ax=ax, color='#1815d1')

for direction in ['top','right','left']:
    ax.spines[direction].set_visible(False)

ax.tick_params(axis='y', labelsize=13)
ax.tick_params(axis='x', labelsize=13)
ax.tick_params(axis = 'y', length=0)  
ax.set_xlabel('Year', fontsize=15, fontweight='bold')
ax.set_ylabel('')
    
plt.show()

# References

- https://www.kaggle.com/subinium/simple-matplotlib-visualization-tips



- https://www.kaggle.com/gaetanlopez/how-to-make-clean-visualizations



- https://www.kaggle.com/vatsalmavani/music-recommendation-system-using-spotify-dataset



- https://en.wikipedia.org/wiki/Pitch_class



- https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/



# This is my first ever Notebook. If you find it useful, give it an upvote.


Also, this notebook is a **Work In Progress**. I will be adding the following soon:

- More Bivariate Analysis

- Genre Analysis

- Code modifcations

Thank you.